In [248]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
import statsmodels.api as sm
import pmdarima as pm
import warnings
from pmdarima.model_selection import train_test_split
from darts import utils
from darts.models import RegressionModel
from darts.models import BlockRNNModel
from darts.models import NaiveSeasonal
from darts.dataprocessing.transformers import Scaler
from darts import TimeSeries
from darts.metrics import mape
from darts.metrics import rmse

In [ ]:
#path to the data folder, edit if necesssary
path = "./data/Generated Data/"

#Price Files 
gas_prices = 'gas_prices.csv'
power_prices = 'power_prices.csv'

#Master files
master_power = 'master_competitor_market_prices_power.csv'
master_gas = 'master_competitor_market_prices_gas.csv'

#Strategy Index aggregated files
power_strat = 'Strat_competitor_market_prices_power.csv'
gas_strat = 'Strat_competitor_market_prices_gas.csv'


In [ ]:
#Reading in relevant datasets
df_power = pd.read_csv(path + master_power)
df_gas = pd.read_csv(path + master_gas)
df_power_strat = pd.read_csv(path + power_strat)
df_gas_strat = pd.read_csv(path + gas_strat)
df_power_prices = pd.read_csv(path + power_prices)
df_gas_prices = pd.read_csv(path + gas_prices)

df_power_strat

In [ ]:
#Setting right indices for price data
df_power_prices.set_index('Dates', drop = True, inplace= True)
df_gas_prices.set_index('date', drop = True, inplace= True)

#Datetime conversion 
df_power_prices.index = pd.to_datetime(df_power_prices.index)
df_gas_prices.index = pd.to_datetime(df_gas_prices.index)


In [ ]:
def querySingleTariffRange(rank = (1,5), post_code = 81737, consumption_range = 2500, df_power = df_power) -> pd.DataFrame:

    res = df_power.loc[(df_power['rank'] >= rank[0]) & (df_power['rank'] <= rank[1]) & (df_power.post_code == post_code) & (df_power.consumption_range_kwh == consumption_range)].copy()
    if res.shape[0] > 0:
        #Generating date indexed data
        res["valid_range"] = res.apply(lambda x: pd.date_range(x["date_valid_from"], x["date_valid_to"]), axis=1)
        res = res.explode("valid_range").copy()

        #Aggregating for mean household prices per day
        res = res.groupby('valid_range').agg({'price_kwh':'mean'})
    return res

In [245]:
def eval_linear_model(model, target, covariates, days = 1, train_size = 0.8):

    #Defining test dataset
    p_train, _ = target.split_before(train_size)
    model.fit(p_train, past_covariates= covariates)
        
    # We backtest the model on the last part of the timeseries:
    backtest = model.historical_forecasts(series=target, 
                                            past_covariates=covariates,
                                            start=train_size, 
                                            retrain=True,
                                            verbose=False, 
                                            forecast_horizon=days)
    return mape(target, backtest)


def evaluate_models(target, covariates, lags, lags_covs, days = 1, train_size = 0.80):
    days = max(days, 1)
    best_score, best_cfg = float("inf"), None
    for l in lags:
        for c in lags_covs:
            try:
                reg_model = RegressionModel(lags = l, lags_past_covariates= c, output_chunk_length=days)
                mse = eval_linear_model(reg_model, target, covariates, days, train_size)
                if mse < best_score:
                    best_score, best_cfg = mse, (l, c)
            except:
                print('error in model eval')
                continue
    print('Best ModelA%s MSE=%.6f' % (best_cfg, best_score))
    return best_cfg

def scaleTimeSeries(timeseries):
    scaler = Scaler()
    series_scaled = scaler.fit_transform(timeseries)
    return series_scaled

In [319]:
def provideForecasts(postcode, consumption_range, rank_range, household_data, price_data, days):

    #Querying dataset
    df = querySingleTariffRange(rank_range, postcode, consumption_range, household_data)
    
    if not df.shape[0]:
        print('No data found.')
        return

    #Shifting prices by 1 to avoid look-ahead bias
    real_prices = df.price_kwh
    df = df.shift(1)
    df.price_kwh = real_prices

    #Merging price data
    df = df.join(price_data)

    #Adding rolling moving average as additional covariate
    df['moving_average'] = df.avg_price.ewm(alpha=0.1, adjust=False).mean()

    #Dropping resulting NA column
    df.dropna(inplace=True)
    
    ##Building TimeSeries objects, and filling in missing date indices
    past_covs = utils.missing_values.fill_missing_values(TimeSeries.from_dataframe(df[['avg_price', 'moving_average']], fill_missing_dates= True))
    prices = utils.missing_values.fill_missing_values(TimeSeries.from_dataframe(df[['price_kwh']], fill_missing_dates= True))

    #Scaling both timeseries
    past_covs = scaleTimeSeries(past_covs)
    prices = scaleTimeSeries(prices)

    #Defining train datasets
    past_covs_train = past_covs[:-days]
    prices_train = prices[:-days]
    prices_valid = prices[-days:]

    #Defining hyperparam grid for linear regression model
    lags = [1,2,3,4,5]
    cov_lags = [1,2,3,4,5]

    #Finding optimal lags
    l, c = evaluate_models(prices_train, past_covs_train, lags, cov_lags, days)

    #Defining Benchmark
    benchmark = NaiveSeasonal(K=1)
    benchmark.fit(prices_train)
    bm = benchmark.predict(days)


    reg_model = RegressionModel(l, c, output_chunk_length=days)
    reg_model.fit(prices_train, past_covs_train)
    rnn_model = BlockRNNModel(input_chunk_length=days, output_chunk_length=days, n_rnn_layers=2, random_state = 42)
    rnn_model.fit(prices_train, past_covariates=past_covs_train, epochs=200, verbose = False)
    preds_rnn = rnn_model.predict(days)
    preds_reg = reg_model.predict(days)


    # prices[-30:].plot(label= 'timeseries')
    # preds_rnn.plot(label = 'predictions RNN')
    # preds_reg.plot(label = 'predictions Regression')
    # bm.plot(label = 'Naive benchmark')

    # print('Baseline MAPE: ', mape(prices[-days:], bm))
    # print('RNN MAPE: ', mape(prices[-days:], preds_rnn))
    # print('Reg MAPE: ', mape(prices[-days:], preds_reg))

    res = preds_reg.values().reshape(1, len(preds_reg))
    res = np.append(res, preds_rnn.values().reshape(1, len(preds_rnn)))
    return res

In [322]:
#Defining postcodes and consumption ranges
df_master = pd.DataFrame({'Postcodes': [], 'Consumption Range': [], 'Rank range': []})
df_res = pd.DataFrame(columns=list(range(1,15))).add_suffix('_linreg')
df_res = pd.concat([df_res, pd.DataFrame(columns=list(range(1,15))).add_suffix('_RNN')])
df_res

#Defining postcodes 
gas_postcodes = df_gas.post_code.unique()[:10]
pow_postcodes = df_power.post_code.unique()[:10]

#Consumption ranges
consumption_ranges = [2500, 3000, 8000]

#Rank ranges
ranks = [(1,5), (10, 15), (20, 25)]

for p in gas_postcodes:
    for c in consumption_ranges:
        for r in ranks:
            print(p, c, r)
            try:
                res = provideForecasts(p, c, r, df_gas, df_gas_prices, 14)
                row = [p, c, r]
                df_master.loc[len(df_master)] = row
                df_res.loc[len(df_res)] = res
            except:
                continue

pd.concat([df_master, df_res], axis = 1)


1309 2500 (1, 5)


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Best ModelA(1, 2) MSE=0.100899


/Users/michaelfuest/DAiA-EON/env/lib/python3.9/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(
`Trainer.fit` stopped: `max_epochs=200` reached.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Predicting: 0it [00:00, ?it/s]

...

Predicting: 0it [00:00, ?it/s]

/var/folders/gm/ndtw23hd4g508cgk1jbt_b640000gn/T/ipykernel_89730/1067548609.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_res.loc[len(df_res)] = res
/var/folders/gm/ndtw23hd4g508cgk1jbt_b640000gn/T/ipykernel_89730/1067548609.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_res.loc[len(df_res)] = res


51381 8000 (1, 5)
No data found.
51381 8000 (10, 15)
No data found.
51381 8000 (20, 25)
No data found.


/var/folders/gm/ndtw23hd4g508cgk1jbt_b640000gn/T/ipykernel_89730/1067548609.py:25: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df_res.loc[len(df_res)] = res


,Postcodes,Consumption Range,Rank range,1_linreg,2_linreg,3_linreg,4_linreg,5_linreg,6_linreg,7_linreg,...,5_RNN,6_RNN,7_RNN,8_RNN,9_RNN,10_RNN,11_RNN,12_RNN,13_RNN,14_RNN
0,1309,2500,"(1, 5)",0.760147,0.758441,0.761269,0.762250,0.761216,0.763759,0.765507,...,0.819896,0.826046,0.834358,0.833642,0.832838,0.823288,0.837080,0.820480,0.825644,0.826005
1,1309,2500,"(10, 15)",0.637183,0.636300,0.635319,0.633898,0.631512,0.627842,0.623535,...,0.654328,0.649229,0.654768,0.654242,0.643917,0.645968,0.642009,0.641710,0.641983,0.643419
2,1309,2500,"(20, 25)",0.687240,0.727376,0.763248,0.789827,0.825058,0.868742,0.895687,...,0.963801,0.963297,0.989403,0.987322,0.989708,0.960349,0.946091,0.907568,0.893425,0.856616
3,1309,3000,"(1, 5)",0.743552,0.739782,0.743410,0.745207,0.743681,0.746031,0.747196,...,0.802063,0.819991,0.817790,0.824614,0.830346,0.811830,0.835487,0.819701,0.815668,0.823977
4,1309,3000,"(10, 15)",0.424287,0.427584,0.430457,0.432796,0.435110,0.437464,0.438345,...,0.422517,0.419823,0.438969,0.432644,0.431819,0.437015,0.422270,0.431999,0.426165,0.421449
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,51381,3000,"(10, 15)",0.425828,0.428943,0.431331,0.433357,0.435719,0.437967,0.438660,...,0.423936,0.422862,0.438766,0.432781,0.434361,0.435970,0.422635,0.434456,0.421616,0.421215
86,51381,3000,"(20, 25)",0.682675,0.713775,0.746239,0.771310,0.794985,0.822176,0.852721,...,0.981931,0.997633,0.998879,1.020365,1.009014,0.971791,0.980772,0.932219,0.896177,0.864773
87,51381,8000,"(1, 5)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,51381,8000,"(10, 15)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
